In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats.mstats import gmean
%matplotlib inline
from subprocess import check_output
print(check_output(["ls", "data/subs/team_subs"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

best_model_per_type.csv
bestpublic.csv
lgbm_1956.csv
mpnn_5_checkpoints_1933.csv
semgcn_1c_w12_24k.csv
stack_median_best_4_submissions.csv
stack_median_lgbm_mpnn_semgcn.csv
stack_median_lgbm_mpnn_semgcn_updated_2080.csv
weighted_average_lgbm_semgcn_mpnn_2001.csv



In [11]:
sub_path = "data/subs/team_subs"

### Best prediction per type

In [12]:
test = pd.read_csv("data/test.csv")
test.head()

id     molecule_name  atom_index_0  atom_index_1  type
0  4658147  dsgdb9nsd_000004             2             0  2JHC
1  4658148  dsgdb9nsd_000004             2             1  1JHC
2  4658149  dsgdb9nsd_000004             2             3  3JHH
3  4658150  dsgdb9nsd_000004             3             0  1JHC
4  4658151  dsgdb9nsd_000004             3             1  2JHC

In [13]:
output_file = 'best_model_per_type_lgbm_mpnn_updated.csv'

df_test = pd.read_csv('data/test.csv')

df_test = df_test.sort_values(by=['id'])

test_id = df_test.id.values
test_type = df_test.type.values

num_test = len(test_id)
coupling_count = np.zeros(num_test)
coupling_value = np.zeros(num_test)

In [14]:
subs = [(['1JHC',  '2JHN', '2JHC', '3JHC'],
         f'{sub_path}/mpnn_5_checkpoints_1933.csv'),
        (['1JHN', '2JHH', '3JHH', '3JHN'],
         f'{sub_path}/lgbm_1956.csv')]

for valid_type, f in subs:
    df = pd.read_csv(f)
    df = df.sort_values(by=['id'])
    for t in valid_type:
        index = np.where(test_type == t)[0]
        coupling_value[index] += df.scalar_coupling_constant.values[index]
        coupling_count[index] += 1
    if len(coupling_count[coupling_count>1]) > 0:
        print('overlap in types!')

coupling_value = coupling_value/coupling_count

df = pd.DataFrame(list(zip(test_id, coupling_value)), columns=['id', 'scalar_coupling_constant'])

### manually looking if submission is correct

In [15]:
test[test.type == '2JHH'].head()

id     molecule_name  atom_index_0  atom_index_1  type
7   4658154  dsgdb9nsd_000015             3             4  2JHH
8   4658155  dsgdb9nsd_000015             3             5  2JHH
11  4658158  dsgdb9nsd_000015             4             5  2JHH
16  4658163  dsgdb9nsd_000015             6             7  2JHH
17  4658164  dsgdb9nsd_000015             6             8  2JHH

In [17]:
mpnn = pd.read_csv(f'{sub_path}/mpnn_5_checkpoints_1933.csv')
mpnn[mpnn.id == 4658154]

id  scalar_coupling_constant
7  4658154                 -7.432317

In [18]:
lgbm = pd.read_csv(f'{sub_path}/lgbm_1956.csv')
lgbm[lgbm.id == 4658154]

id  scalar_coupling_constant
7  4658154                 -7.484122

In [19]:
df[df.id == 4658154] #should be from lgbm

id  scalar_coupling_constant
7  4658154                 -7.484122

In [21]:
test[test.type == '3JHH'].head()

id     molecule_name  atom_index_0  atom_index_1  type
2   4658149  dsgdb9nsd_000004             2             3  3JHH
27  4658174  dsgdb9nsd_000016             3             5  3JHH
28  4658175  dsgdb9nsd_000016             3             6  3JHH
29  4658176  dsgdb9nsd_000016             3             7  3JHH
30  4658177  dsgdb9nsd_000016             3             8  3JHH

In [22]:
mpnn = pd.read_csv(f'{sub_path}/mpnn_5_checkpoints_1933.csv')
mpnn[mpnn.id == 4658149]

id  scalar_coupling_constant
2  4658149                  1.869468

In [23]:
lgbm = pd.read_csv(f'{sub_path}/lgbm_1956.csv')
lgbm[lgbm.id == 4658149]

id  scalar_coupling_constant
2  4658149                 10.244528

In [24]:
df[df.id == 4658149] #should be from lgbm

id  scalar_coupling_constant
2  4658149                 10.244528

In [25]:
test[test.type == '1JHC'].head()

id     molecule_name  atom_index_0  atom_index_1  type
1   4658148  dsgdb9nsd_000004             2             1  1JHC
3   4658150  dsgdb9nsd_000004             3             0  1JHC
5   4658152  dsgdb9nsd_000015             3             0  1JHC
9   4658156  dsgdb9nsd_000015             4             0  1JHC
12  4658159  dsgdb9nsd_000015             5             0  1JHC

In [26]:
mpnn = pd.read_csv(f'{sub_path}/mpnn_5_checkpoints_1933.csv')
mpnn[mpnn.id == 4658148]

id  scalar_coupling_constant
1  4658148                174.137238

In [27]:
lgbm = pd.read_csv(f'{sub_path}/lgbm_1956.csv')
lgbm[lgbm.id == 4658148]

id  scalar_coupling_constant
1  4658148                199.016079

In [29]:
df[df.id == 4658148] #should be from mpnn

id  scalar_coupling_constant
1  4658148                174.137238

In [30]:
df[['id', 'scalar_coupling_constant']].to_csv(output_file, index=False, float_format='%.6f')